In [ ]:
!pip install pandas
!pip install fuzzywuzzy

import pandas as pd
from fuzzywuzzy.fuzz import partial_ratio

In [ ]:
# produce risk score based on price data
grouped_price = pd.read_csv('./datasets/aggregated-details/grouped_details.csv')
price_cols = [x for x in grouped_price if 'price' in x]


price_details = pd.DataFrame(columns=['num_matches', 'num_jumps', 'max_change'])
for company in grouped_price['company_name']:
    print(company)   
    fuzzy_matches = [ind for ind in grouped_price.index if partial_ratio(company, grouped_price['company_name'][ind])>=90]

    matched_rows = grouped_price.loc[fuzzy_matches][price_cols]
    mean_prices = matched_rows.mean(axis=0).dropna()

    num_jumps = max_change = 0
    prev_price = mean_prices[0]
    for price in mean_prices[1:]:
        if prev_price != price:
            num_jumps += 1
            change = abs((price - prev_price)/prev_price)

            if  change > max_change:
                max_change = change
        
        prev_price = price

    price_details = price_details.append({
        'num_matches': len(fuzzy_matches), 
        'num_jumps': num_jumps,
        'max_change': max_change*100
    }, ignore_index=True)


price_details['company_name'] = grouped_price['company_name']
price_details.to_csv('./datasets/indicators/price_details.csv', index=False)